In [1]:
def dropout(image, DIM=256, PROBABILITY = 0.75, CT = 8, SZ = 0.2):
    # input - one image of size [dim,dim,3] not a batch of [b,dim,dim,3]
    # output - image with CT squares of side size SZ*DIM removed

    # DO DROPOUT WITH PROBABILITY DEFINED ABOVE
    P = tf.cast( tf.random.uniform([],0,1) < PROBABILITY, tf.int32)
    if (P == 0)|(CT == 0)|(SZ == 0): return image

    for k in range( CT ):
        # CHOOSE RANDOM LOCATION
        x = tf.cast( tf.random.uniform([],0,DIM),tf.int32)
        y = tf.cast( tf.random.uniform([],0,DIM),tf.int32)
        # COMPUTE SQUARE 
        WIDTH = tf.cast( SZ*DIM,tf.int32) * P
        ya = tf.math.maximum(0,y-WIDTH//2)
        yb = tf.math.minimum(DIM,y+WIDTH//2)
        xa = tf.math.maximum(0,x-WIDTH//2)
        xb = tf.math.minimum(DIM,x+WIDTH//2)
        # DROPOUT IMAGE
        one = image[ya:yb,0:xa,:]
        two = tf.zeros([yb-ya,xb-xa,3]) 
        three = image[ya:yb,xb:DIM,:]
        middle = tf.concat([one,two,three],axis=1)
        image = tf.concat([image[0:ya,:,:],middle,image[yb:DIM,:,:]],axis=0)

    # RESHAPE HACK SO TPU COMPILER KNOWS SHAPE OF OUTPUT TENSOR 
    image = tf.reshape(image,[DIM,DIM,3])
    return image

In [2]:
def sigmoid(x):
    return 1 / (1+np.exp(-x))

In [3]:
df_scores = pd.read_csv('../output/scores.csv')

exp_name = 'benchmark_exp_2'
df_exp = df_scores.loc[df_scores['experiment'] == exp_name]
df_exp

,epoch,loss,auc_score,val_loss,val_auc_score,elapsed_time,lr,best_val_auc_step,best_val_loss_step,best_val_loss_step_val_auc,best_val_auc_step_val_loss,best_val_auc_step_auc,best_val_auc_step_loss,experiment,model_name,fold,ckpt,cv_folds,backbone,sampler,optim,lr.1,pool,output_net,scheduler,scheduler_params,criterion,criterion_params,batch_size,class_weight,norm_cols,max_norm,fp_16,random_state,val_roc_auc_score,val_accuracy_score,val_precision_score,val_recall_score,val_f1_score,val_average_precision_score,val_specificity_score,val_fpr_score,val_pos_rate,dt
0,12.0,0.647094,0.976014,0.629296,0.920502,426.0,0.000298,12,13,0.887996,0.629296,0.976014,0.647094,benchmark_exp_2,efficientnet-b1_bce_smth_224_64_concat_adam_one_cycle_20200804_1123,2,ckpt_0012_0.629296_0.920502.pt,cv_folds_20200802_0048,efficientnet-b1,random,adam,0.00001,{'method': 'concat'},"{'bn': {'eps': 0.001, 'momentum': 0.01}, 'dropout': 0.3}",one_cycle,{'max_lr': 0.0005},bce_smth,"{'smoothing': 0.03, 'class_weight': 'balanced'}",64,False,['source'],3.0,False,42069,0.920502,0.846503,0.092827,0.830189,0.166983,0.353612,0.846812,0.153188,0.213888,2020-08-04 11:23:00
1,5.0,0.811875,0.941760,0.635931,0.919883,306.0,0.000470,5,6,0.908282,0.635931,0.941760,0.811875,benchmark_exp_2,efficientnet-b0_bce_smth_224_64_concat_adam_one_cycle_20200804_2010,1,ckpt_0005_0.635931_0.919883.pt,cv_folds_20200802_0048,efficientnet-b0,random,adam,0.00001,{'method': 'concat'},"{'bn': {'eps': 0.001, 'momentum': 0.01}, 'dropout': 0.3}",one_cycle,{'max_lr': 0.0005},bce_smth,"{'smoothing': 0.03, 'class_weight': 'balanced'}",64,False,['source'],1.5,False,42069,0.919883,0.834007,0.084586,0.882353,0.154374,0.224592,0.833162,0.166838,0.167393,2020-08-04 20:10:00
2,5.0,0.817412,0.943169,0.608070,0.911670,306.0,0.000470,5,5,0.911670,0.608070,0.943169,0.817412,benchmark_exp_2,efficientnet-b0_bce_smth_224_64_concat_adam_one_cycle_20200804_2220,2,ckpt_0005_0.608070_0.911670.pt,cv_folds_20200802_0048,efficientnet-b0,random,adam,0.00001,{'method': 'concat'},"{'bn': {'eps': 0.001, 'momentum': 0.01}, 'dropout': 0.3}",one_cycle,{'max_lr': 0.0005},bce_smth,"{'smoothing': 0.03, 'class_weight': 'balanced'}",64,False,['source'],1.5,False,42069,0.911670,0.903497,0.139159,0.811321,0.237569,0.349048,0.905237,0.094763,0.193442,2020-08-04 22:20:00
7,7.0,0.780188,0.956497,0.623119,0.904883,427.0,0.000433,7,7,0.904883,0.623119,0.956497,0.780188,benchmark_exp_2,efficientnet-b1_bce_smth_224_64_concat_adam_one_cycle_20200804_0821,1,ckpt_0007_0.623119_0.904883.pt,cv_folds_20200802_0048,efficientnet-b1,random,adam,0.00001,{'method': 'concat'},"{'bn': {'eps': 0.001, 'momentum': 0.01}, 'dropout': 0.3}",one_cycle,{'max_lr': 0.0005},bce_smth,"{'smoothing': 0.03, 'class_weight': 'balanced'}",64,False,['source'],3.0,False,42069,0.904883,0.809428,0.071547,0.843137,0.131902,0.237535,0.808839,0.191161,0.228239,2020-08-04 08:21:00
8,15.0,0.791069,0.943065,0.638010,0.904439,164.0,0.000204,15,16,0.894028,0.638010,0.943065,0.791069,benchmark_exp_2,resnet34_bce_smth_224_64_concat_adam_one_cycle_20200804_1538,1,ckpt_0015_0.638010_0.904439.pt,cv_folds_20200802_0048,resnet34,random,adam,0.00001,{'method': 'concat'},"{'bn': {'eps': 0.001, 'momentum': 0.01}, 'dropout': 0.3}",one_cycle,{'max_lr': 0.0005},bce_smth,"{'smoothing': 0.03, 'class_weight': 'balanced'}",64,False,['source'],3.0,False,42069,0.904439,0.752862,0.061617,0.941176,0.115663,0.208513,0.749572,0.250428,0.197348,2020-08-04 15:38:00
19,16.0,0.763281,0.955884,0.617192,0.896882,160.0,0.000173,16,16,0.896882,0.617192,0.955884,0.763281,benchmark_exp_2,resnet34_bce_smth_224_64_concat_adam_one_cycle_20200804_1649,2,ckpt_0016_0.617192_0.896882.pt,cv_folds_20200802_0048,resnet34,random,adam,0.00001,{'method': 'concat'},"{'bn': {'eps': 0.001, 'momentum': 0.01}, 'dropout': 0.3}",one_cycle,{'max_lr': 0.0005},bce_smth,"{'smoothing': 0.03, 'class_weight': 'balanced'}",64,False,['source'],3.0,False,42069,0.896882,0.873077,0.104592,0.773585,0.184270,0.278187,0.874955,0.125045,0.220555,2020-

In [ ]:
df_scores.lo

In [4]:
model_name = 'resnet34_bce_smth_224_64_concat_adam_one_cycle_20200802_1544'
df_model = df_scores.loc[df_scores['model_name'] == model_name]

In [8]:
df_scores['delta'] = df_scores['best_val_auc_step_auc'] - df_scores['val_roc_auc_score']

In [10]:
df_scores.loc[df_scores['experiment']== 'sampler_exp_1']

,epoch,loss,auc_score,val_loss,val_auc_score,elapsed_time,lr,best_val_auc_step,best_val_loss_step,best_val_loss_step_val_auc,best_val_auc_step_val_loss,best_val_auc_step_auc,best_val_auc_step_loss,experiment,model_name,fold,ckpt,cv_folds,backbone,sampler,optim,lr.1,pool,output_net,scheduler,scheduler_params,criterion,criterion_params,batch_size,class_weight,norm_cols,max_norm,fp_16,random_state,val_roc_auc_score,val_accuracy_score,val_precision_score,val_recall_score,val_f1_score,val_average_precision_score,val_specificity_score,val_fpr_score,val_pos_rate,dt,delta
13,17.0,0.609415,0.951449,0.460355,0.901586,164.0,0.000144,17,12,0.889665,0.460355,0.951449,0.609415,sampler_exp_1,resnet34_bce_smth_224_64_concat_adam_one_cycle_20200804_0203,0,ckpt_0017_0.460355_0.901586.pt,cv_folds_20200802_0048,resnet34,random,adam,0.00001,{'method': 'concat'},"{'bn': {'eps': 0.001, 'momentum': 0.01}, 'dropout': 0.3}",one_cycle,{'max_lr': 0.0005},bce_smth,"{'smoothing': 0.01, 'class_weight': 'balanced'}",64,False,"['source', 'anatom_site_general_challenge']",1.5,False,42069,0.901586,0.728769,0.054147,0.94,0.102397,0.172361,0.725234,0.274766,0.117663,2020-08-04 02:03:00,0.049863
22,10.0,0.870168,0.924592,0.658356,0.895134,164.0,0.000357,10,12,0.872811,0.658356,0.924592,0.870168,sampler_exp_1,resnet34_bce_smth_224_64_concat_adam_one_cycle_20200804_0052,0,ckpt_0010_0.658356_0.895134.pt,cv_folds_20200802_0048,resnet34,random,adam,0.00001,{'method': 'concat'},"{'bn': {'eps': 0.001, 'momentum': 0.01}, 'dropout': 0.3}",one_cycle,{'max_lr': 0.0005},bce_smth,"{'smoothing': 0.03, 'class_weight': 'balanced'}",64,False,"['source', 'anatom_site_general_challenge']",1.5,False,42069,0.895134,0.777485,0.060393,0.86,0.112861,0.119726,0.776104,0.223896,0.239657,2020-08-04 00:52:00,0.029458
31,10.0,0.582763,0.969945,0.589781,0.880783,164.0,0.000357,10,15,0.816218,0.589781,0.969945,0.582763,sampler_exp_1,resnet34_bce_smth_224_64_concat_adam_one_cycle_20200803_1735,0,ckpt_0010_0.589781_0.880783.pt,cv_folds_20200802_0048,resnet34,imbalanced,adam,0.00001,{'method': 'concat'},"{'bn': {'eps': 0.001, 'momentum': 0.01}, 'dropout': 0.3}",one_cycle,{'max_lr': 0.0005},bce_smth,"{'smoothing': 0.01, 'class_weight': 'balanced'}",64,False,"['source', 'anatom_site_general_challenge']",1.5,False,42069,0.880783,0.796577,0.063077,0.82,0.117143,0.191517,0.796185,0.203815,0.156890,2020-08-03 17:35:00,0.089162
34,6.0,0.849139,0.946006,0.792328,0.871365,164.0,0.000453,6,15,0.753273,0.792328,0.946006,0.849139,sampler_exp_1,resnet34_bce_smth_224_64_concat_adam_one_cycle_20200803_2231,0,ckpt_0006_0.792328_0.871365.pt,cv_folds_20200802_0048,resnet34,imbalanced,adam,0.00001,{'method': 'concat'},"{'bn': {'eps': 0.001, 'momentum': 0.01}, 'dropout': 0.3}",one_cycle,{'max_lr': 0.0005},bce_smth,"{'smoothing': 0.03, 'class_weight': 'balanced'}",64,False,"['source', 'anatom_site_general_challenge']",1.5,False,42069,0.871365,0.769585,0.054795,0.80,0.102564,0.235640,0.769076,0.230924,0.272945,2020-08-03 22:31:00,0.074641
37,16.0,0.616345,0.956636,0.478939,0.863260,164.0,0.000173,16,16,0.863260,0.478939,0.956636,0.616345,sampler_exp_1,resnet34_bce_smth_224_64_concat_adam_one_cycle_20200803_1554,0,ckpt_0016_0.478939_0.863260.pt,cv_folds_20200802_0048,resnet34,batch,adam,0.00001,{'method': 'concat'},"{'bn': {'eps': 0.001, 'momentum': 0.01}, 'dropout': 0.3}",one_cycle,{'max_lr': 0.0005},bce_smth,"{'smoothing': 0.01, 'class_weight': 'balanced'}",64,False,"['source', 'anatom_site_general_challenge']",1.5,False,42069,0.863260,0.758065,0.054616,0.84,0.102564,0.187900,0.756693,0.243307,0.116342,2020-08-03 15:54:00,0.093376
39,19.0,0.715740,0.967735,0.665833,0.860207,164.0,0.000091,19,14,0.838447,0.665833,0.967735,0.715740,sampler_exp_1,resnet34_bce_smth_224_64_concat_adam_one_cycle_20200803_2342,0,ckpt_0019_0.665833_0.860207.pt,cv_folds_20200802_0048,resnet34,batch,adam,0.00001,{'method': 'concat'},"{'bn': {'eps': 0.001, 'momentum': 0.01}, 'dropout': 0.3}",one_cycle,{'max_lr': 0.0005},bce_smth,"{'smoothing': 0

In [9]:
df_scores.sort_values('delta')

,epoch,loss,auc_score,val_loss,val_auc_score,elapsed_time,lr,best_val_auc_step,best_val_loss_step,best_val_loss_step_val_auc,best_val_auc_step_val_loss,best_val_auc_step_auc,best_val_auc_step_loss,experiment,model_name,fold,ckpt,cv_folds,backbone,sampler,optim,lr.1,pool,output_net,scheduler,scheduler_params,criterion,criterion_params,batch_size,class_weight,norm_cols,max_norm,fp_16,random_state,val_roc_auc_score,val_accuracy_score,val_precision_score,val_recall_score,val_f1_score,val_average_precision_score,val_specificity_score,val_fpr_score,val_pos_rate,dt,delta
35,19.0,0.051525,0.415981,0.084123,0.865919,464.0,1.730523e-06,19,12,0.862673,0.084123,0.415981,0.051525,arch_exp_1,efficientnet-b4_bce_224_32_concat_adam_one_cycle_20200730_1049,0,ckpt_0019_0.084123_0.865919.pt,cv_folds_20200728_0112,efficientnet-b4,batch,adam,1.000000e-07,{'method': 'concat'},NaN,one_cycle,{'max_lr': 5e-06},bce,{},32,False,['anatom_site_general_challenge'],1.5,False,42069,0.865919,0.738663,0.059653,0.859375,0.111562,0.190396,0.736314,0.263686,0.010906,2020-07-30 10:49:00,-0.449938
9,12.0,1.204616,0.617195,1.198224,0.903684,176.0,4.126004e-04,12,12,0.903684,1.198224,0.617195,1.204616,bn_exp_1,efficientnet-b0_bce_smth_224_64_concat_adam_one_cycle_20200731_2241,0,ckpt_0012_1.198224_0.903684.pt,cv_folds_20200728_0112,efficientnet-b0,random,adam,1.000000e-05,{'method': 'concat'},"{'bn': {'eps': 0.001, 'momentum': 0.01}, 'dropout': 0.03}",one_cycle,{'max_lr': 0.0005},bce_smth,"{'smoothing': 0.01, 'class_weight': 'balanced'}",64,False,['anatom_site_general_challenge'],3.0,False,42069,0.903684,0.812351,0.081481,0.859375,0.148850,0.198150,0.811436,0.188564,0.369186,2020-07-31 22:41:00,-0.286489
11,40.0,0.065181,0.622889,0.071154,0.902558,162.0,7.702631e-06,40,34,0.901146,0.071154,0.622889,0.065181,efficientnet_lr_exp_1,efficientnet-b0_bce_224_64_concat_adam_one_cycle_20200730_1722,0,ckpt_0040_0.071154_0.902558.pt,cv_folds_20200728_0112,efficientnet-b0,random,adam,1.000000e-04,{'method': 'concat'},NaN,one_cycle,{'max_lr': 0.005},bce,{},64,False,['anatom_site_general_challenge'],3.0,False,42069,0.896817,0.776551,0.071339,0.890625,0.132097,0.135190,0.774331,0.225669,0.017564,2020-07-30 17:22:00,-0.273928
12,40.0,0.065181,0.622889,0.071154,0.902558,162.0,7.702631e-06,40,34,0.901146,0.071154,0.622889,0.065181,lr_exp_1,efficientnet-b0_bce_224_64_concat_adam_one_cycle_20200730_1454,0,ckpt_0040_0.071154_0.902558.pt,cv_folds_20200728_0112,efficientnet-b0,random,adam,1.000000e-04,{'method': 'concat'},NaN,one_cycle,{'max_lr': 0.005},bce,{},64,False,['anatom_site_general_challenge'],3.0,False,42069,0.896817,0.776551,0.071339,0.890625,0.132097,0.135190,0.774331,0.225669,0.017564,2020-07-30 14:54:00,-0.273928
25,9.0,2.225111,0.623658,2.268195,0.892531,162.0,4.524600e-04,9,11,0.878241,2.268195,0.623658,2.225111,misc_exp_1,efficientnet-b0_bce_smth_224_64_concat_adam_one_cycle_20200731_1820,0,ckpt_0009_2.268195_0.892531.pt,cv_folds_20200728_0112,efficientnet-b0,random,adam,1.000000e-05,{'method': 'concat'},NaN,one_cycle,{'max_lr': 0.0005},bce_smth,"{'smoothing': 0.1, 'class_weight': 'balanced'}",64,False,['anatom_site_general_challenge'],3.0,False,42069,0.892531,0.789379,0.075397,0.890625,0.139024,0.191464,0.787409,0.212591,0.761735,2020-07-31 18:20:00,-0.268873
20,19.0,0.061466,0.643420,0.074027,0.896194,87.0,2.893339e-04,19,18,0.894892,0.074027,0.643420,0.061466,lr_exp_1,resnet34_bce_224_64_concat_adam_one_cycle_20200729_2139,0,ckpt_0019_0.074027_0.896194.pt,cv_folds_20200728_0112,resnet34,random,adam,1.000000e-05,{'method': 'concat'},NaN,one_cycle,{'max_lr': 0.0005},bce,{},64,False,['anatom_site_general_challenge'],5.0,False,42069,0.908403,0.731205,0.062696,0.937500,0.117532,0.176703,0.727190,0.272810,0.015818,2020-07-29 21:39:00,-0.264983
4,12.0,0.027458,0.667382,0.084748,0.907452,87.0,4.126004e-05,12,4,0.899749,0.084748,0.667382,0.027458,lr_exp_1,resnet34_bce_224_64_concat_adam_one_cycle_20200729_2037,0,ckpt_0012_0.084748_0.907452.pt,cv_folds_20200728_0112,resnet34,ran

In [5]:
df_model

,epoch,loss,auc_score,val_loss,val_auc_score,elapsed_time,lr,best_val_auc_step,best_val_loss_step,best_val_loss_step_val_auc,best_val_auc_step_val_loss,best_val_auc_step_auc,best_val_auc_step_loss,experiment,model_name,fold,ckpt,cv_folds,backbone,sampler,optim,lr.1,pool,output_net,scheduler,scheduler_params,criterion,criterion_params,batch_size,class_weight,norm_cols,max_norm,fp_16,random_state,val_roc_auc_score,val_accuracy_score,val_precision_score,val_recall_score,val_f1_score,val_average_precision_score,val_specificity_score,val_fpr_score,val_pos_rate,dt
27,10.0,0.874268,0.922894,0.6514,0.888969,164.0,0.000357,10,16,0.863273,0.6514,0.922894,0.874268,ext_data_exp_1,resnet34_bce_smth_224_64_concat_adam_one_cycle_20200802_1544,0,ckpt_0010_0.651400_0.888969.pt,cv_folds_20200802_0048,resnet34,random,adam,0.00001,{'method': 'concat'},"{'bn': {'eps': 0.001, 'momentum': 0.01}, 'dropout': 0.3}",one_cycle,{'max_lr': 0.0005},bce_smth,"{'smoothing': 0.03, 'class_weight': 'balanced'}",64,False,['source'],3.0,False,42069,0.888969,0.793943,0.06231,0.82,0.115819,0.127524,0.793507,0.206493,0.241228,2020-08-02 15:44:00


In [18]:
df_model

,epoch,loss,auc_score,val_loss,val_auc_score,elapsed_time,lr,best_val_auc_step,best_val_loss_step,best_val_loss_step_val_auc,best_val_auc_step_val_loss,best_val_loss_step_auc,best_val_auc_step_loss,experiment,model_name,fold,ckpt,cv_folds,backbone,sampler,optim,lr.1,pool,output_net,scheduler,scheduler_params,criterion,criterion_params,batch_size,class_weight,norm_cols,max_norm,fp_16,random_state,val_roc_auc_score,val_accuracy_score,val_precision_score,val_recall_score,val_f1_score,val_average_precision_score,val_specificity_score,val_fpr_score,val_pos_rate,dt
28,10.0,0.874268,0.922894,0.6514,0.888969,164.0,0.000357,10,16,0.863273,0.6514,0.951615,0.874268,ext_data_exp_1,resnet34_bce_smth_224_64_concat_adam_one_cycle_20200802_1544,0,ckpt_0010_0.651400_0.888969.pt,cv_folds_20200802_0048,resnet34,random,adam,0.00001,{'method': 'concat'},"{'bn': {'eps': 0.001, 'momentum': 0.01}, 'dropout': 0.3}",one_cycle,{'max_lr': 0.0005},bce_smth,"{'smoothing': 0.03, 'class_weight': 'balanced'}",64,False,['source'],3.0,False,42069,0.888969,0.793943,0.06231,0.82,0.115819,0.127524,0.793507,0.206493,0.241228,2020-08-02 15:44:00


In [4]:
exp_name = 'benchmark_exp_2'
df_exp = df_scores.loc[df_scores['experiment'] ==  exp_name]

In [5]:
model_dir = '../models'
model_name = 'efficientnet-b1_bce_smth_224_64_concat_adam_one_cycle_20200804_1123'
fold = 2
fold_dir = os.path.join(model_dir, exp_name, model_name, f'fold_{fold}')
df_hist = pd.read_csv(os.path.join(fold_dir, 'history.csv'))

In [26]:
df_hist = df_hist.rename(columns={'val_roc_auc_score': 'val_auc_score'}) 

In [27]:
df_hist.loc[df_hist['epoch'] == 13, 'val_auc_score']

12    0.887996
Name: val_auc_score, dtype: float64

In [6]:
model_dir = '../models'
model_name = 'efficientnet-b1_bce_smth_224_64_concat_adam_one_cycle_20200804_1123'
fold = 2
fold_dir = os.path.join(model_dir, exp_name, model_name, f'fold_{fold}')
df_hist = pd.read_csv(os.path.join(fold_dir, 'history.csv'))
df_val = pd.read_csv(os.path.join(fold_dir, 'val_predictions.csv'))

In [7]:
df_model = df_scores.loc[df_scores['model_name'] == model_name]
df_model

,epoch,loss,auc_score,val_loss,val_auc_score,elapsed_time,lr,best_val_auc_step,best_val_loss_step,best_val_loss_step_auc,best_val_auc_step_loss,experiment,model_name,fold,ckpt,cv_folds,backbone,sampler,optim,lr.1,pool,output_net,scheduler,scheduler_params,criterion,criterion_params,batch_size,class_weight,norm_cols,max_norm,fp_16,random_state,val_roc_auc_score,val_accuracy_score,val_precision_score,val_recall_score,val_f1_score,val_average_precision_score,val_specificity_score,val_fpr_score,dt
0,12.0,0.647094,0.976014,0.629296,0.920502,426.0,0.000298,12,13,0.887996,0.629296,benchmark_exp_2,efficientnet-b1_bce_smth_224_64_concat_adam_one_cycle_20200804_1123,2,ckpt_0012_0.629296_0.920502.pt,cv_folds_20200802_0048,efficientnet-b1,random,adam,0.00001,{'method': 'concat'},"{'bn': {'eps': 0.001, 'momentum': 0.01}, 'dropout': 0.3}",one_cycle,{'max_lr': 0.0005},bce_smth,"{'smoothing': 0.03, 'class_weight': 'balanced'}",64,False,['source'],3.0,False,42069,0.920502,0.846503,0.092827,0.830189,0.166983,0.353612,0.846812,0.153188,2020-08-04 11:23:00


In [9]:
from melanoma.utils import train_utils
METRICS = [                                                                                                                                                                                                                    
    'roc_auc_score',                                                                                                                                                                                                           
    'accuracy_score',                                                                                                                                                                                                              'precision_score',                                                                                                                                                                                                         
    'recall_score',                                                                                                                                                                                                            
    'f1_score',                                                                                                                                                                                                                    'average_precision_score',                                                                                                                                                                                                 
    'specificity_score',                                                                                                                                                                                                       
    'fpr_score',
    'pos_rate'
]

In [10]:
train_utils.compute_scores(df_val['target'], df_val['prediction_raw'], METRICS)

{'roc_auc_score': 0.9205019795524665,
 'accuracy_score': 0.8465034965034965,
 'precision_score': 0.09282700421940929,
 'recall_score': 0.8301886792452831,
 'f1_score': 0.16698292220113853,
 'average_precision_score': 0.35361199156773365,
 'specificity_score': 0.8468115425721411,
 'fpr_score': 0.15318845742785892,
 'pos_rate': 0.21388763742668504}

In [43]:
df_hist.loc[df_hist['val_roc_auc_score'].idxmax()]

epoch                           12.000000
loss                             0.647094
auc_score                        0.976014
val_loss                         0.629296
elapsed_time                   426.000000
lr                               0.000298
val_roc_auc_score                0.920502
val_accuracy_score               0.846503
val_precision_score              0.092827
val_recall_score                 0.830189
val_f1_score                     0.166983
val_average_precision_score      0.353612
val_specificity_score            0.000000
val_fpr_score                    0.000000
Name: 11, dtype: float64

In [49]:
df_hist.loc[df_hist['val_loss'].idxmin()]

epoch                           13.000000
loss                             0.624167
auc_score                        0.983949
val_loss                         0.616165
elapsed_time                   425.000000
lr                               0.000266
val_roc_auc_score                0.887996
val_accuracy_score               0.901049
val_precision_score              0.136076
val_recall_score                 0.811321
val_f1_score                     0.233062
val_average_precision_score      0.415558
val_specificity_score            0.000000
val_fpr_score                    0.000000
Name: 12, dtype: float64

In [ ]:
def load_his